# Fine tune CDLA pretrained model for KR court decisions

In [1]:
import subprocess, os, time, json
import fitz
import paddle
paddle.utils.run_check()

!export CUDA_VISIBLE_DEVICES=0

Running verify PaddlePaddle program ... 


I0428 19:04:48.897567 1489050 program_interpreter.cc:212] New Executor is Running.
W0428 19:04:48.898406 1489050 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0428 19:04:48.906631 1489050 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.


PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)


I0428 19:04:49.432123 1489050 interpreter_util.cc:624] Standalone Executor is Used.


In [2]:
print(os.getcwd())

/home/lstm/github/PaddleDetection/tuning


## Train

In [29]:
# Single GPU
command = (
    "python3 ../tools/train.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_test.yml",
    "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    "--eval",
    f"--use_vdl=True --vdl_log_dir=vdl_log/{time.strftime('%Y%m%d-%H%M%S')}",
    )
command = " ".join(command)
print(command)

python3 ../tools/train.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_test.yml --slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml --eval --use_vdl=True --vdl_log_dir=vdl_log/20240428-172301


In [31]:
!{command}
# output = subprocess.run(command, shell=True, capture_output=True)

/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W0428 17:25:03.925673 1421608 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0428 17:25:03.927322 1421608 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
[04/28 17:25:04] ppdet.utils.checkpoint INFO: The shape [42] in pretrained weight head.head_cls0.bias is unmatched with the shape [44] in model head.head_cls0.bias. And the weight head.head_cls0.bias will not be loaded
[04/28 17:25:04] ppdet.utils.checkpoint INFO: The shape [42, 128, 1, 1] in pretrained weight head.head_cls0.weight is unmatched with the shape [44, 128, 1, 1] in model head.head_cls0.weight. And the weight head.head_cls0.weight will not be loaded
[04/28 17:25:04] ppdet.utils.checkpoint INFO: The shape [42] in pretrained weight head.head_cls1.bias is unmatched with the shape [44] in model head.head_cls1.bias. And the w

In [ ]:
# run visualdl in shell (from ./tuning dir)
# visualdl --logdir ./vdl_log/20240428-041209 --port 7070

## Tune

In [ ]:
# -r output/faster_rcnn_r50_1x_coco/10000

## Eval

In [ ]:

command = (
    "python3 ../tools/eval.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    # "-o weights=../output/best_model",
    "-o weights=../output/0.pdparams",
    # "use_gpu=true",
    "--use_vdl=True --vdl_log_dir=vdl_log"
    )
command = " ".join(command)
print(command)

In [ ]:
!{command}

## Test 
##### Strategy: use command line method to test trained weights; create inference model for production.

#### Test prep (convert PDFs to images)

In [ ]:
# convert pdf to image and save

import fitz
import os

pdf_folder = './testdata/'
# pdf_folder = './tuning/testdata/'

for pdf_path in os.listdir(pdf_folder):

    if not pdf_path.endswith('.pdf'):
        continue

    pdf_name = os.path.splitext(pdf_path)[0]
    img_folder = os.path.join(pdf_folder, pdf_name)
    if not os.path.isdir(img_folder):
        os.mkdir(img_folder)

    pdf_path = os.path.join(pdf_folder, pdf_path)
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        img_name = os.path.join(img_folder, f'{page_num}.png')
        if not os.path.isfile(img_name):
            page.get_pixmap().save(img_name)
        else:
            print(f"{img_name} exists!")            

    doc.close()

#### Test inference

In [3]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"
# opts = "weights=./trained_weights/output_20240423/best_model/model.pdparams"

inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'

inference_outputfolder = './testoutput'
testout_img_folder = 'adobe_대법원_2020도1007_판결서'

In [4]:
args = {
    'config': configfile,
    'slim_config': slim_configfile,
    'infer_dir': f'{inference_datafolder}/{testdoc_img_folder}',
    'infer_img': None,
    'output_dir': f'{inference_datafolder}/{testout_img_folder}',
    'draw_threshold': 0.5, # default    
    'use_vdl': True,
    'vdl_log_dir': '/vdl_log_test',
    'save_results': False,
    # slice_infer, action='store_true',
    # slice_size, nargs='+,
    # overlap_ratio, nargs='+',
    # combine_method: None,
    # match_threshold: None,
    # match_metric: None,
    'visualize': True
}

print(json.dumps(args, indent=4))

{
    "config": "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    "slim_config": " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    "infer_dir": "./testdata/adobe_\ub300\ubc95\uc6d0_2020\ub3c41007_\ud310\uacb0\uc11c",
    "infer_img": null,
    "output_dir": "./testdata/adobe_\ub300\ubc95\uc6d0_2020\ub3c41007_\ud310\uacb0\uc11c",
    "draw_threshold": 0.5,
    "use_vdl": true,
    "vdl_log_dir": "/vdl_log_test",
    "save_results": false,
    "visualize": true
}


In [5]:
datapath = 'testdata'
filelist = os.listdir(datapath)

def pdf2byte(filename):
    pdf = fitz.open(os.path.join(datapath, filename))
    for pg in range(pdf.pageCount):
        # convert pdf to image and store in local variable
        page = pdf[pg]
        pix = page.getPixmap()
        img_data = page.getPixmap().tobytes("png")
        # stor pix in local variable


In [28]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_test.yml"
slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"

inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'
weight2use = '20240428'
output_dir = './testresults'

command = (
    "python3 ../tools/infer.py",
    f"-c {configfile}",    
    f"--slim_config {slim_configfile}",
    f"--infer_dir={inference_datafolder}/{testdoc_img_folder}",
    # f"--infer_img={inference_datafolder}/{testdoc_img_folder}/0.png",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    "--draw_threshold=0.5",
    # "use_gpu=true",
    "--use_vdl=True",
    "--vdl_log_dir=./vdl_log_test",
    f"--output_dir={output_dir}"
    )
command = " ".join(command)
print(command)

!{command}



                "-c",
                "./configs/picodet/legacy_model/application/layout_analysis/CDLA.yml", 
                "--slim_config",
                "./configs/picodet/legacy_model/application/layout_analysis/picodet_lcnet_x2_5_layout.yml",
                "--infer_dir=./tuning/testdata/adobe_대법원_2020도1007_판결서",
                "-o",
                "weights=./tuning/trained_weights/output_CDLApretrained/model.pdparams",
                "--draw_threshold=0.5",
                "--use_vdl=True",
                "--vdl_log_dir=./tuning/vdl_log_test",
                "--output_dir=./tuning/testresults"
            ]



python3 ../tools/infer.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_test.yml --slim_config  ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml --infer_dir=./testdata/adobe_대법원_2020도1007_판결서 -o weights=./trained_weights/output_20240428/best_model/model.pdparams --draw_threshold=0.5 --use_vdl=True --vdl_log_dir=./vdl_log_test --output_dir=./testresults
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W0428 16:00:08.365183 1361093 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0428 16:00:08.367453 1361093 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
[04/28 16:00:09] ppdet.utils.checkpoint INFO: Finish loading model weights: ./trained_weights/output_20240428/best_model/model.pdparams
[04/28 16:00:09] train INFO: Found 4 inference images in total.
libpng error: IDA

In [26]:
f"{inference_datafolder}/{testdoc_img_folder}"

'./testdata/adobe_대법원_2020도1007_판결서'

## Deploy (use PaddleOCR)

#### Model export (distilled model)
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/ppstructure/docs/quickstart_en.md

In [ ]:
model_outpath = './exported_model'

command = (
    "python3 ../tools/export_model.py",
    f"-c {configfile}",    
    f"--slim_config {slim_configfile}",    
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",    
    f"--output_dir={model_outpath}"
    )
command = " ".join(command)
print(command)


In [ ]:
!{command}

#### Model inference (distilled model)

| Field | Description | Default |
|-|-|-|  
| output | result save path | ./output/table |
| table_max_len | long side of the image resize in table structure model | 488 |
| table_model_dir | Table structure model inference model path | None |
| table_char_dict_path | The dictionary path of table structure model | ../ppocr/utils/dict/table_structure_dict.txt |
| merge_no_span_structure | In the table recognition model, whether to merge '<td>' and '</td>' | False |
| layout_model_dir | Layout analysis model inference model path | None |
| layout_dict_path | The dictionary path of layout analysis model | ../ppocr/utils/dict/layout_publaynet_dict.txt |
| layout_score_threshold | The box threshold path of layout analysis model | 0.5 |
| layout_nms_threshold | The nms threshold path of layout analysis model | 0.5 |
| kie_algorithm | kie model algorithm | LayoutXLM |
| ser_model_dir | Ser model inference model path | None |
| ser_dict_path | The dictionary path of Ser model | ../train_data/XFUND/class_list_xfun.txt |
| mode | structure or kie | structure |
| image_orientation | Whether to perform image orientation classification in forward | False |
| layout | Whether to perform layout analysis in forward | True |
| table | Whether to perform table recognition in forward | True |
| ocr | Whether to perform ocr for non-table areas in layout analysis. When layout is False, it will be automatically set to False | True |
| recovery | Whether to perform layout recovery in forward | False |
| save_pdf | Whether to convert docx to pdf when recovery | False |
| structure_version | Structure version, optional PP-structure and PP-structurev2 | PP-structure |


In [ ]:
# using PPStrcuture class

import os
import cv2
from paddleocr import PPStructure, save_structure_res


weight2use = '20240423'
output_dir = './inference_output'
layout_dict_path = "./labels/labels_10.txt"

table_engine = PPStructure(
    output=output_dir,
    table_max_len=None, # 488 
    table_model_dir=None,
    table_char_dict_path=None, # ../ppocr/utils/dict/table_structure_dict.txt
    merge_no_span_structure=False,
    layout_model_dir=model_outpath, # None
    layout_dict_path=layout_dict_path,
    layout_score_threshold=0.5, # 0.5
    layout_nms_threshold= 0.3, # 0.5
    kie_algorithm=None, # LayoutXLM
    ser_model_dir=None,
    ser_dict_path=None, #../train_data/XFUND/class_list_xfun.txt
    mode="structure",
    image_orientation=False,
    layout=True,
    table=False,
    ocr=False,
    recovery=False,
    save_pdf=False,
    structure_version="PP-structurev2",
    show_log=True
    )

inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'

save_folder = f'{output_dir}/visual'
img_path = ''
# img_dir = f'{inference_datafolder}/{testdoc_img_folder}'
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])

for line in result:
    line.pop('img')
    print(line)

In [ ]:
# # usin predict_system.py
# from 

# python3 predict_system.py --layout_model_dir=inference/picodet_lcnet_x1_0_fgd_layout_cdla_infer \
#                           --image_dir=../DATA/sample_data/판례/png/image61_0.png \
#                           --output=../output \
#                           --table=false \
#                           --ocr=false \
#                           --vis_font_path=../NotoSansKR-Regular.otf \
#                           --layout_dict_path ../ppocr/utils/dict/layout_dict/layout_cdla_dict.txt